In [1]:
import torch
import torchvision
from torchvision.transforms import transforms
from visdom import Visdom
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import time
import numpy as np

In [2]:
transform_train = transforms.Compose(
[
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

transform_test = transforms.Compose(
[
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

train_dataset = torchvision.datasets.CIFAR10('./data',train=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10('./data',train=False, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 128,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 128,shuffle=False)



In [3]:
viz = Visdom()

In [4]:
classes = ['plane','car','bird','cat','deer','dog','frog','horse','ship','truck']
dataiter = iter(test_loader)
image,label = dataiter.next()
image = viz.images(image[:10]/2+0.5,nrow=10,padding=3)
text=viz.text('||'.join('%6s' % classes[label[j]] for j in range(10)))

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(3,16,5,1,2),
                                   nn.BatchNorm2d(16),
                                   nn.ReLU(inplace=True))
                                   
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Sequential(nn.Conv2d(16,32,3,1,1),
                                   nn.BatchNorm2d(32),
                                   nn.ReLU(inplace=True))
        self.fc1 = nn.Sequential(nn.Linear(32*8*8,256),
                                 nn.BatchNorm1d(256),
                                 nn.ReLU(inplace=True))
        self.fc2 = nn.Sequential(nn.Linear(256,128),
                                  nn.BatchNorm1d(128),
                                  nn.ReLU(inplace=True))
        self.fc3 = nn.Linear(128,10)
    def forward(self,x):
        x = self.pool(self.conv1(x))
        x = self.pool(self.conv2(x))
        x = x.view(-1,32*8*8)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x
net = Net()
if torch.cuda.is_available:
    net = net.cuda()

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr=0.001,momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=30,gamma=0.1)

In [7]:
start_time = time.time()
line = viz.line(Y=np.arange(10))
tr_loss, ts_loss,tr_acc,ts_acc,step=[],[],[],[],[]
best_acc = 0
best_state = net.state_dict()

In [8]:
for epoch in range(90):
    running_loss, running_acc,img_num = 0.0,0.0,0
    scheduler.step()
    net.train()
    for i,(img,label) in enumerate(train_loader,1):
        if torch.cuda.is_available:
            img, label = img.cuda(),label.cuda()
        img,label = Variable(img),Variable(label)
        output = net(img)
        loss = criterion(output,label)
        pred = torch.max(output,1)[1]
        #print(pred)
        running_acc += sum(pred==label).data[0]
        running_loss += loss.data[0]*len(label)
        #print(running_loss)
        img_num += len(label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(img_num)
    print("Train: [{}/{}] | loss:{:.4f} | r_acc :{:.4f}".format(epoch+1,200,running_loss/img_num,running_acc/img_num))
    tr_loss.append(running_loss/img_num)
    tr_acc.append(running_acc/img_num)
          
    net.eval()
    eval_loss,eval_acc,img_num=0.0,0.0,0
    for i,(img,label) in enumerate(test_loader,1):
        if torch.cuda.is_available:
            img, label = img.cuda(),label.cuda()
        img,label = Variable(img),Variable(label)
        output = net(img)
        loss = criterion(output,label)
        pred = torch.max(output,1)[1]
        eval_acc += sum(pred==label).data[0]
        eval_loss += loss.data[0]*len(label)
        img_num += len(label)
          
        
    print("Val: [{}/{}] | loss:{:.4f} | r_acc :{:.4f}".format(epoch+1,200,eval_loss/img_num,eval_acc/img_num))
    ts_loss.append(eval_loss/img_num)
    ts_acc.append(eval_acc/img_num)
          
    viz.line(Y=np.column_stack((np.array(tr_loss),np.array(tr_acc),np.array(ts_loss),np.array(ts_acc))),
            win=line,
            opts=dict(legend=["tr_loss","tr_acc","ts_loss","ts_acc"],title="cafar10"),)
    if eval_acc/img_num > best_acc:
          best_acc = eval_acc/img_num
          best_state=net.state_dict()

50000
Train: [1/200] | loss:1.8230 | r_acc :0.3845
Val: [1/200] | loss:1.5443 | r_acc :0.4814
50000
Train: [2/200] | loss:1.4065 | r_acc :0.5192
Val: [2/200] | loss:1.2939 | r_acc :0.5526
50000
Train: [3/200] | loss:1.2185 | r_acc :0.5777
Val: [3/200] | loss:1.1575 | r_acc :0.5945
50000
Train: [4/200] | loss:1.1007 | r_acc :0.6184
Val: [4/200] | loss:1.0909 | r_acc :0.6223
50000
Train: [5/200] | loss:1.0100 | r_acc :0.6507
Val: [5/200] | loss:1.0508 | r_acc :0.6269
50000
Train: [6/200] | loss:0.9376 | r_acc :0.6761
Val: [6/200] | loss:0.9608 | r_acc :0.6666
50000
Train: [7/200] | loss:0.8827 | r_acc :0.6952
Val: [7/200] | loss:0.9291 | r_acc :0.6756
50000
Train: [8/200] | loss:0.8329 | r_acc :0.7105
Val: [8/200] | loss:0.9082 | r_acc :0.6816
50000
Train: [9/200] | loss:0.7920 | r_acc :0.7255
Val: [9/200] | loss:0.8674 | r_acc :0.6973
50000
Train: [10/200] | loss:0.7546 | r_acc :0.7400
Val: [10/200] | loss:0.8700 | r_acc :0.6966
50000
Train: [11/200] | loss:0.7159 | r_acc :0.7550
Val: [

KeyboardInterrupt: 